In [ ]:
%load_ext autoreload
%autoreload 2


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['font.sans-serif'] = "Ubuntu"
plt.rcParams['font.weight'] = "light"
plt.rcParams["figure.facecolor"] = "white"

import covid19

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr

In [ ]:
COPYRIGHT_EPIDEMICS = "\xa9 2021 Alessandro Amici, dati github.com/pcm-dpc/COVID-19"
COPYRIGHT_VACCINES = "\xa9 2021 Alessandro Amici, dati github.com/italia/covid19-opendata-vaccini"

In [ ]:
!git -C ../../../pcm-dpc/COVID-19 pull

In [ ]:
data_italy_path = '../../../pcm-dpc/COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv'

data_italy = covid19.data.read_dpc(data_italy_path, location_prefix="")

data_italy = data_italy.drop_vars(["lat", "lon", "country", "state_region"])

data_italy_sum = data_italy.sum('location').expand_dims(location=['Italia']).assign_coords(population=data_italy.population.sum("location").expand_dims(location=['Italia']))
data_italy = xr.merge([data_italy, data_italy_sum])
data_italy = data_italy.assign_coords(population=data_italy.population.astype(int))

for data_var_name in ["deaths", "tests"]:
    data_italy["daily_" + data_var_name] = data_italy[data_var_name].diff("time")
    data_italy = data_italy.drop_vars(data_var_name)

data_italy = data_italy.fillna(0).astype(int)

for data_var_name in data_italy:
    if data_var_name.startswith("daily_"):
        data_italy[data_var_name + "7"] = data_italy[data_var_name].rolling(time=7).mean()

data_italy["daily_critical7"] = xr.where(data_italy.time > np.datetime64("2021-01-01"), data_italy["daily_critical7"], np.nan)

data_italy_last_day = data_italy.time.values[-1]
data_italy_first_day = data_italy.time.values[0]

data_italy

In [ ]:
data_italy.isel(time=-1).to_dataframe()

In [ ]:
data_italy_week_ago = data_italy.shift(time=7)
print((data_italy - data_italy_week_ago).daily_confirmed.isel(time=slice(-14, None)).sel(location="Italia").to_series())
print((data_italy - data_italy_week_ago).daily_tests.isel(time=slice(-14, None)).sel(location="Italia").to_series())
print(data_italy.daily_tests.isel(time=slice(-14, None)).sel(location="Italia").to_series())

In [ ]:
data_italy.sel(location="Italia").daily_confirmed[data_italy.sel(location="Italia").daily_confirmed >= 10172].isel(time=slice(-14, None)).to_series()

In [ ]:
data_italy_pm = data_italy / data_italy.population * 1_000_000

data_italy_pm["tpr"] = data_italy_pm.daily_confirmed / data_italy_pm.daily_tests
data_italy_pm["tpr7"] = data_italy_pm.daily_confirmed.rolling(time=7).sum() / data_italy_pm.daily_tests.rolling(time=7).sum()

data_italy_pm

In [ ]:
data_italy_pm.sel(location="Italia").isel(time=slice(-14, None)).to_dataframe()

In [ ]:
COLORS = list(sns.color_palette())
KINDS = ["daily_tests7", "daily_confirmed7", "current_severe", "current_critical", "daily_deaths7", "tpr7", None, None, "daily_critical7"]
DAY = np.timedelta64(1, "D")

XLIM_ALL = (data_italy_first_day, data_italy_last_day + 40 * DAY)
XLIM_ONE_YEAR = (data_italy_last_day - (365 + 30) * DAY, data_italy_last_day + 30 * DAY)
XLIM_THREE_MONTHS = (data_italy_last_day - 100 * DAY, data_italy_last_day + 20 * DAY)
XLIM_FOUR_MONTHS = (data_italy_last_day - 130 * DAY, data_italy_last_day + 20 * DAY)
XLIM_SIX_MONTHS = (data_italy_last_day - 20 * DAY - 182 * DAY, data_italy_last_day + 20 * DAY)
XLIM_THREE_MONTHS_LAST_YEAR = (data_italy_last_day - (100 + 365) * DAY, data_italy_last_day + (20 - 365) * DAY)


In [ ]:
SHOW = None

data = data_italy_pm.sel(location="Italia")

fig, ax = covid19.plot.subplots(subplot_kw={'yscale': 'log', 'ylim': (0.07, 15000), "xlim": XLIM_ALL}, figsize=(12, 10), note=COPYRIGHT_EPIDEMICS, note_pos=(1, 0.02))

for kind, color in reversed(list(zip(KINDS, COLORS))):
    if kind is None or (SHOW and kind not in SHOW):
        continue
    covid19.plot.plot_data(ax, data[kind], label=kind, color=color, date_interval=35, marker=None, annotate=True)

_ = ax.set_title(f'COVID-19 Italia - dati Protezione Civile al {str(data_italy_last_day)[:10]}')
_ = ax.set(xlabel="", ylabel="# per milione di abitanti")
_ = ax.legend(loc='upper left')

In [ ]:
SHOW = {"daily_confirmed7", "current_severe", "current_critical", "daily_deaths7", "daily_critical7"}

data = data_italy_pm.sel(location="Italia")

fig, ax = covid19.plot.subplots(subplot_kw={'yscale': 'log', 'ylim': (5, 500), "xlim": XLIM_SIX_MONTHS}, figsize=(12, 10), note=COPYRIGHT_EPIDEMICS, note_pos=(1, 0.02))

SCALE = [0.3, .5, 0.7, 5.5, 50, 1, 1, 1, 70]

for kind, color, scale in zip(KINDS, COLORS, SCALE):
    if kind is None or (SHOW and kind not in SHOW):
        continue
    label = kind if scale == 1 else f"{kind} times {scale}"
    covid19.plot.plot_data(ax, data[kind], label=label, color=color, date_interval=35, marker=None, annotate=True, ratio=1 / scale)

_ = ax.set_title(f'COVID-19 Italia - dati Protezione Civile al {str(data_italy_last_day)[:10]}')
_ = ax.set(xlabel="", ylabel="# per milione di abitanti")
_ = ax.legend(loc='upper left')

In [ ]:
SHOW = {"daily_confirmed7", "current_severe", "current_critical", "daily_deaths7", "daily_critical7"}

data = data_italy_pm.sel(location="Italia")

fig, ax = covid19.plot.subplots(subplot_kw={'yscale': 'log', 'ylim': (0.07, 150), "xlim": XLIM_THREE_MONTHS}, figsize=(12, 10), note=COPYRIGHT_EPIDEMICS, note_pos=(1, 0.02))

for kind, color in reversed(list(zip(KINDS, COLORS))):
    if kind is None or (SHOW and kind not in SHOW):
        continue
    covid19.plot.plot_data(ax, data[kind], label=kind, color=color, date_interval=7, marker=None, annotate=True)

_ = ax.set_title(f'COVID-19 Italia - dati Protezione Civile al {str(data_italy_last_day)[:10]}')
_ = ax.set(xlabel="", ylabel="# per milione di abitanti")
_ = ax.legend(loc='upper left')

In [ ]:
SHOW = {"daily_confirmed7"}
FIT_DAYS = 60

data = data_italy_pm.sel(location="Italia")

fig, ax = covid19.plot.subplots(subplot_kw={'yscale': 'log', 'ylim': (10, 800), "xlim": XLIM_SIX_MONTHS}, figsize=(12, 10), note=COPYRIGHT_EPIDEMICS, note_pos=(1, 0.02))

SCALE = [0.3, 1, 1.3, 10, 130, 1, 1, 1, 132]
DELAY = [None, None, None, None, None, None, None, None, None]

for kind, color, scale, delay in zip(KINDS, COLORS, SCALE, DELAY):
    if kind is None or (SHOW and kind not in SHOW):
        continue
    label = kind if scale == 1 else f"{kind} times {scale}"
    fit = covid19.fit.fit_exponential_segments(data[kind], breaks=[data_italy_last_day - FIT_DAYS * DAY, None], min_value=0, valid_ratio=0.1)
    if fit:
        ff = fit[0]
        if abs(ff.T_d_days) < FIT_DAYS:
            covid19.plot.plot_fit(ax, ff, color=color, extrapolate=[-2, 30], marker=None, label=f"{kind} {FIT_DAYS}-days fit", ratio=1 / scale)
    covid19.plot.plot_data(ax, data[kind], label=label, color=color, date_interval=28, marker=None, annotate=True, ratio=1 / scale, delay=delay)

_ = ax.set_title(f'COVID-19 Italia - dati Protezione Civile al {str(data_italy_last_day)[:10]}')
_ = ax.set(xlabel="", ylabel="# per milione di abitanti")
_ = ax.legend(loc='lower left')

In [ ]:
SHOW = {"daily_confirmed7"}
FIT_DAYS = 11
HIGHLIGHT_DAY = 1

data = data_italy.sel(location="Italia")

fig, ax = covid19.plot.subplots(subplot_kw={'ylim': (0, 35000), "xlim": XLIM_SIX_MONTHS}, figsize=(12, 10), note=COPYRIGHT_EPIDEMICS, note_pos=(1, 0.02))

SCALE = [0.3, 1, 1.3, 10, 130, 1, 1, 1, 132]
DELAY = [None, None, None, None, None, None, None, None, None]

for kind, color, scale, delay in zip(KINDS, COLORS, SCALE, DELAY):
    if kind is None or (SHOW and kind not in SHOW):
        continue
    label = 'nuovi casi giornalieri' if scale == 1 else f"{kind} times {scale}"
    ax.bar(data.time, data[kind[:-1]], color=color, alpha=0.25, label=label)
    ax.bar(data.time[HIGHLIGHT_DAY::7], data[kind[:-1]][HIGHLIGHT_DAY::7], color=color, alpha=0.25)
    covid19.plot.plot_data(ax, data[kind], label=label + " (media su 7 giorni)", color=color, date_interval=28, marker=None, annotate=True, ratio=1 / scale, delay=delay)
    fit = covid19.fit.fit_exponential_segments(data[kind], breaks=[data_italy_last_day - FIT_DAYS * DAY, None], min_value=0, valid_ratio=0.1)
    if fit:
        ff = fit[0]
        if True or abs(ff.T_d_days) < FIT_DAYS:
            covid19.plot.plot_fit(ax, ff, color=color, extrapolate=[-2, 30], marker=None, label=f"andamento dei {label}", ratio=1 / scale)

_ = ax.set_title(f'COVID-19 Italia - dati Protezione Civile al {str(data_italy_last_day)[:10]}')
_ = ax.set(xlabel="", ylabel="")
_ = ax.legend(loc='upper left')

In [ ]:
SHOW = {"daily_confirmed7"}

data = data_italy_pm.sel(location="Italia")

fig, ax = covid19.plot.subplots(subplot_kw={'yscale': 'log', 'ylim': (30, 150), "xlim": XLIM_THREE_MONTHS}, figsize=(12, 10), note=COPYRIGHT_EPIDEMICS, note_pos=(1, 0.02))

SCALE = [0.3, 1, 1.3, 10, 130, 1, 1, 1, 132]
DELAY = [None, None, None, None, None, None, None, None, None]

for kind, color, scale, delay in zip(KINDS, COLORS, SCALE, DELAY):
    if kind is None or (SHOW and kind not in SHOW):
        continue
    label = kind if scale == 1 else f"{kind} times {scale}"
    fit = covid19.fit.fit_exponential_segments(data[kind], breaks=["2021-10-19", "2021-10-22"], min_value=0, valid_ratio=0)
    print(fit)
    if fit:
        ff = fit[0]
        if True  or abs(ff.T_d_days) < FIT_DAYS:
            covid19.plot.plot_fit(ax, ff, color=color, extrapolate=[-2, 30], marker=None, label=f"{kind} 3-days fit", ratio=1 / scale)
    covid19.plot.plot_data(ax, data[kind], label=label, color=color, marker=None, date_interval=7, annotate=True, ratio=1 / scale, delay=delay)

_ = ax.set_title(f'COVID-19 Italia - dati Protezione Civile al {str(data_italy_last_day)[:10]}')
_ = ax.set(xlabel="", ylabel="# per milione di abitanti")
_ = ax.legend(loc='upper left')

In [ ]:
SHOW = {"tpr7"}

data = data_italy_pm.sel(location="Italia")

fig, ax = covid19.plot.subplots(subplot_kw={'yscale': 'log', 'ylim': (0.005, 0.05), "xlim": XLIM_SIX_MONTHS}, figsize=(12, 10), note=COPYRIGHT_EPIDEMICS, note_pos=(1, 0.02))

SCALE = [0.3, 1, 1.3, 5.5, 75, 1, 1, 1, 140]
DELAY = [None, None, None, None, None, None, None, None, -4]

for kind, color, scale, delay in zip(KINDS, COLORS, SCALE, DELAY):
    if kind is None or (SHOW and kind not in SHOW):
        continue
    label = kind if scale == 1 else f"{kind} times {scale}"
    fit = covid19.fit.fit_exponential_segments(data[kind], breaks=[data_italy_last_day - FIT_DAYS * DAY, None], min_value=0, valid_ratio=0.1)
    if fit:
        ff = fit[0]
        if True or abs(ff.T_d_days) < FIT_DAYS:
            covid19.plot.plot_fit(ax, ff, color=color, extrapolate=[-2, 30], marker=None, label=f"{kind} {FIT_DAYS}-days fit", ratio=1 / scale)
    covid19.plot.plot_data(ax, data[kind], label=label, color=color, date_interval=7, marker=None, annotate=True, ratio=1 / scale, delay=delay)

_ = ax.set_title(f'COVID-19 Italia - dati Protezione Civile al {str(data_italy_last_day)[:10]}')
_ = ax.set(xlabel="", ylabel="# per milione di abitanti")
_ = ax.legend(loc='upper left')

In [ ]:
SHOW = {"daily_confirmed7", "current_severe", "current_critical", "daily_deaths7", "daily_critical7"}

data = data_italy_pm.sel(location="Italia")

fig, ax = covid19.plot.subplots(subplot_kw={'yscale': 'log', 'ylim': (0.07, 150), "xlim": XLIM_THREE_MONTHS_LAST_YEAR}, figsize=(12, 10), note=COPYRIGHT_EPIDEMICS, note_pos=(1, 0.02))

for kind, color in zip(KINDS, COLORS):
    if kind is None or (SHOW and kind not in SHOW):
        continue
    covid19.plot.plot_data(ax, data[kind], label=kind, color=color, date_interval=7, marker=None, annotate=False)

_ = ax.set_title(f'COVID-19 Italia - dati Protezione Civile al {str(data_italy_last_day)[:10]}')
_ = ax.set(xlabel="", ylabel="# per milione di abitanti")
_ = ax.legend(loc='upper left')

In [ ]:
!git -C ../../../italia/covid19-opendata-vaccini pull

In [ ]:
vaccines_italy_path = '../../../italia/covid19-opendata-vaccini/dati/somministrazioni-vaccini-latest.csv'

vaccines_italy = covid19.data.read_vaccini(vaccines_italy_path)
vaccines_italy

In [ ]:
providers = vaccines_italy.doses.sum(["time", "age_class", "location", "dose_type"])
providers

In [ ]:
doses = vaccines_italy.doses.sortby(-providers)

primed = doses.sel(dose_type="first")
vaccinated = xr.where(doses.provider == "Janssen", doses.sel(dose_type="first"), doses.sel(dose_type="second"))
boosted = doses.sel(dose_type="booster")

In [ ]:
location = "Italia"
fig, ax = covid19.plot.subplots(note=COPYRIGHT_VACCINES)
covid19.plot.stack_xarray(primed.sel(location=location).sum("age_class"), hue="provider", window=7, title=f"Popolazione - {location}", label_total="Media a 7 gioni del totale", ax=ax)
ax.set(ylabel="prime dosi somministrate al giorno")
ax.legend()

In [ ]:
location = "Italia"
fig, ax = covid19.plot.subplots(note=COPYRIGHT_VACCINES)
covid19.plot.stack_xarray(doses.sel(location=location).sum(["provider", "age_class"]), hue="dose_type", window=7, title=f"Popolazione - {location}", label_total="Media a 7 gioni del totale", ax=ax, date_interval=45)
ax.set(ylabel="dosi somministrate al giorno")
ax.legend()


In [ ]:
location = "Italia"
data = primed.sel(location=location).cumsum("time").sum("age_class") / primed.sel(location=location).population
fig, ax = covid19.plot.subplots(note=COPYRIGHT_VACCINES)
covid19.plot.stack_xarray(data, hue="provider", title=f"Popolazione - almeno una dose - {location}", ylim=(0, 1), alpha=0.8, ax=ax)
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
ax.set(ylabel="% popolazione")
ax.legend() 

In [ ]:
location = "Italia"
data = vaccinated.sel(location=location).cumsum("time").sum("age_class") / vaccinated.sel(location=location).population
fig, ax = covid19.plot.subplots(note=COPYRIGHT_VACCINES)
covid19.plot.stack_xarray(data, hue="provider", title=f"Popolazione - vaccinati - {location}", ylim=(0, 1), alpha=0.8, ax=ax)
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
ax.set(ylabel="% popolazione")
ax.legend() 

In [ ]:
location = "Italia"
boosted_full = boosted.sel(location=location).cumsum("time").sum(["age_class", "provider"])
vaccinated_full = vaccinated.sel(location=location).cumsum("time").sum(["age_class", "provider"])
primed_full = primed.sel(location=location).cumsum("time").sum(["age_class", "provider"])
data = xr.concat([
    boosted_full.expand_dims(status=["booster"]),
    (vaccinated_full - boosted_full).expand_dims(status=["second dose"]),
    (primed_full - vaccinated_full).expand_dims(status=["first dose"])
], dim="status", coords='minimal', compat='override')

data = data / primed.sel(location=location).population

fig, ax = covid19.plot.subplots(note=COPYRIGHT_VACCINES)
covid19.plot.stack_xarray(data, hue="status", title=f"Popolazione - vaccinati - {location}", ylim=(0, 1), alpha=0.8, ax=ax, date_interval=42)
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1, 0))
ax.set(ylabel="% popolazione")
ax.legend(loc="upper left")


In [ ]:
SHOW = {"daily_confirmed7", "daily_critical7"}
SCALE = [0.3, 1, 0.7, 5, 50, 1, 1, 1, 110]
YLIM = (1 * 0.9, 1000 / 0.9)
FIT_DAYS = 42
SHOW_FIT = SHOW

for location in data_italy.location.values:
    data = data_italy_pm.sel(location=location)
    _, ax = covid19.plot.subplots(subplot_kw={'xlim': XLIM_ONE_YEAR, 'yscale': 'log'}, note=COPYRIGHT_EPIDEMICS, note_pos=(1, 0.01))

    for kind, color, scale in zip(KINDS, COLORS, SCALE):
        if kind is None or (SHOW and kind not in SHOW):
            continue
        label = kind if scale == 1 else f"{kind} times {scale}"
        covid19.plot.plot_data(ax, data[kind], label=label, color=color, date_interval=21, marker=None, annotate=True, ratio=1 / scale, ylim=[10, 1000])
        if kind in SHOW_FIT:
            fit = covid19.fit.fit_exponential_segments(data[kind], breaks=[data_italy_last_day - FIT_DAYS * DAY, None], min_value=0, valid_ratio=0.1)
            if fit:
                ff = fit[0]
                if abs(ff.T_d_days) < FIT_DAYS:
                    covid19.plot.plot_fit(ax, ff, color=color, extrapolate=[-2, 30], marker=None, label=f"{kind} {FIT_DAYS}-days fit", ratio=1 / scale)

    _ = ax.set_title(f'COVID-19 {location} - dati Protezione Civile al {str(data_italy_last_day)[:10]}')
    _ = ax.set(xlabel="", ylabel="# per milione di abitanti")
    _ = ax.legend(loc='lower center')
